Implementation of the tutorial found at http://pyro.ai/examples/intro_part_ii.html, using inference in Pyro, with my comments

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

In this example we are trying to figure out how much something weighs, but the scale we use gives slightly different answers every time we weigh the same object. 

We compensate for these noisy measurements by incorporating prior knowledge into a probability distribution which should describe how the weights provided by the scale vary. 


This is encoded by the following two distributions, 

weight | guess ~ Normal(guess, 1), and measurement | guess, weight ~ Normal(weight, .75)


Where guess is what our prior estimate of the object's weight is, so the true weight is going to be a gaussian distribution of unit variance centered on our guess. Additionally, the measurement of the scale in turn is a combination of these two things, a normal distribution centered on the true weight with slightly less variance than what our guess was. 

In [3]:
def scale(guess):
    """
    The distributions described above can be written in pyro as: 
    
    """
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75))

# Conditioning

So far we haven't done anything special, but where Pyro separates us is that it allows us to condition generative models on observations, and infer latent factors by what might have produced that data (MCMC preview). In Pyro, conditioning is separate from it's evaluation, meaning it is possible to make a model and condition it on multiple things at once. So for our problem above, the observed data may come in many forms, and Pyro allows us to infer the other latent factors agnostic of where we start. 



As an example, consider we guess that the weight of an object is 8.5, but observe a measurement of 9.5. Accordingly, we wish to infer the distribution (weight | guess, measurement = 9.5) ~ ? (So given our prior and measurement, what is the true weight? 

Pyro allows us to do this with conditioned. 


In [4]:
conditioned_scale = pyro.condition(scale, data={"measurement": 9.5})


In [6]:
conditioned_scale



_bound_partial(functools.partial(<function _context_wrap at 0x2b9124800790>, <pyro.poutine.condition_messenger.ConditionMessenger object at 0x2b9124c969d0>, <function scale at 0x2b90b21773a0>))

In [7]:
def deferred_conditioned_scale(measurement, guess):
    """
    behaves just like an ordinary Python function, 
    conditioning can be deferred or parametrized with Python’s lambda or def
    """
    return pyro.condition(scale, data={"measurement": measurement})(guess)

In [8]:
def scale_obs(guess):  # equivalent to conditioned_scale above
    """
    Now we sample on a conditional distribution, where the observation was given below.
    """
    weight = pyro.sample("weight", dist.Normal(guess, 1.))
     # here we condition on measurement == 9.5
    return pyro.sample("measurement", dist.Normal(weight, 0.75), obs=9.5) 

In [14]:
scale_obs(0) # returns the same thing agnostic of guess? 

# This makes sense I guess. Only one observation, we should center our weight and therefore guess around it. 

9.5

http://www.stat.cmu.edu/~brian/463-663/week09/Chapter%2003.pdf

In [18]:
def perfect_guide(guess):
    """
    An inference algorithm for computing the posterior, what our new weight estimate is given a guess and measurement. 
    """
    loc =(0.75**2 * guess + 9.5) / (1 + 0.75**2) # 9.14
    scale = np.sqrt(0.75**2/(1 + 0.75**2)) # 0.6
    return pyro.sample("weight", dist.Normal(loc, scale))

In [24]:
perfect_guide(-1000) #played around with this a bit... 

tensor(-355.2377)

That's all for now, but here's another cool tutorial

http://pyro.ai/examples/vae.html